### Deep Learning Example: Transfer Learning using AlexNet and CIFAR-10 Dataset
Copyright 2017 The MathWorks, Inc.
You will need to download images in order to run this example.
Please see the file in this directory: DownloadCIFAR10.m Running this file will help you download CIFAR10 if you choose to use those images.

### Bring in Alex Net
If this is your first time running this code, this may cause an error message to occur. Follow the link to download AlexNet. You'll only need to download alexnet once.

In [1]:
net = alexnet;

### Access Layers
Visualize the layers of AlexNet, what do we see about this architecture? What do we have to change for this to work for our new data?

In [2]:
layers = net.Layers;

layers


layers = 

  25x1 Layer array with layers:

     1   'data'     Image Input                   227x227x3 images with 'zerocenter' normalization
     2   'conv1'    Convolution                   96 11x11x3 convolutions with stride [4  4] and padding [0  0  0  0]
     3   'relu1'    ReLU                          ReLU
     4   'norm1'    Cross Channel Normalization   cross channel normalization with 5 channels per element
     5   'pool1'    Max Pooling                   3x3 max pooling with stride [2  2] and padding [0  0  0  0]
     6   'conv2'    Grouped Convolution           2 groups of 128 5x5x48 convolutions with stride [1  1] and padding [2  2  2  2]
     7   'relu2'    ReLU                          ReLU
     8   'norm2'    Cross Channel Normalization   cross channel normalization with 5 channels per element
     9   'pool2'    Max Pooling                   3x3 max pooling with stride [2  2] and padding [0  0  0  0]
    10   'conv3'    Convolution                   384 3x3x256 conv

## Train
### Set up training data

In [4]:
rootFolder = 'cifar10Train';
categories = {'deer','dog','frog','cat'};
imds = imageDatastore(fullfile(rootFolder, categories), 'LabelSource', 'foldernames');

imds = splitEachLabel(imds, 500, 'randomize') % we only need 500 images per class
imds.ReadFcn = @readFunctionTrain;


imds = 

  ImageDatastore with properties:

                       Files: {
                              ' .../Github/matlab-paths/DeepLearning/cifar10Train/deer/image10024.png';
                              ' .../Github/matlab-paths/DeepLearning/cifar10Train/deer/image10068.png';
                              ' .../Github/matlab-paths/DeepLearning/cifar10Train/deer/image10106.png'
                               ... and 1997 more
                              }
                      Labels: [deer; deer; deer ... and 1997 more categorical]
    AlternateFileSystemRoots: {}
                    ReadSize: 1
                     ReadFcn: @readDatastoreImage



### Take layers from Alex Net, then add our own

In [5]:
layers = layers(1:end-3);

layers(end+1) = fullyConnectedLayer(64, 'Name', 'special_2');
layers(end+1) = reluLayer;
layers(end+1) = fullyConnectedLayer(4, 'Name', 'fc8_2 ');
layers(end+1) = softmaxLayer;
layers(end+1) = classificationLayer()


layers = 

  27x1 Layer array with layers:

     1   'data'        Image Input                   227x227x3 images with 'zerocenter' normalization
     2   'conv1'       Convolution                   96 11x11x3 convolutions with stride [4  4] and padding [0  0  0  0]
     3   'relu1'       ReLU                          ReLU
     4   'norm1'       Cross Channel Normalization   cross channel normalization with 5 channels per element
     5   'pool1'       Max Pooling                   3x3 max pooling with stride [2  2] and padding [0  0  0  0]
     6   'conv2'       Grouped Convolution           2 groups of 128 5x5x48 convolutions with stride [1  1] and padding [2  2  2  2]
     7   'relu2'       ReLU                          ReLU
     8   'norm2'       Cross Channel Normalization   cross channel normalization with 5 channels per element
     9   'pool2'       Max Pooling                   3x3 max pooling with stride [2  2] and padding [0  0  0  0]
    10   'conv3'       Convolution     

### Fine-tune learning rates [advanced]

In [6]:
layers(end-2).WeightLearnRateFactor = 10;
layers(end-2).WeightL2Factor = 1;
layers(end-2).BiasLearnRateFactor = 20;
layers(end-2).BiasL2Factor = 0;

### Other training options

In [7]:
opts = trainingOptions('sgdm', ...
    'LearnRateSchedule', 'none',...
    'InitialLearnRate', .0001,... 
    'MaxEpochs', 20, ...
    'MiniBatchSize', 128);

### Train! 
Please note this may take a few minutes or longer depending on hardware. Training with a GPU is strongly encouraged.

In [8]:
convnet = trainNetwork(imds, layers, opts);

Training on single GPU.
Initializing input data normalization.
|========================================================================================|
|  Epoch  |  Iteration  |  Time Elapsed  |  Mini-batch  |  Mini-batch  |  Base Learning  |
|         |             |   (hh:mm:ss)   |   Accuracy   |     Loss     |      Rate       |
|========================================================================================|
|       1 |           1 |       00:00:05 |       36.72% |       2.3009 |      1.0000e-04 |
|       4 |          50 |       00:02:51 |       59.38% |       1.0367 |      1.0000e-04 |
|       7 |         100 |       00:06:22 |       71.88% |       0.6788 |      1.0000e-04 |
|      10 |         150 |       00:07:28 |       78.12% |       0.5960 |      1.0000e-04 |
|      14 |         200 |       00:09:40 |       84.38% |       0.4406 |      1.0000e-04 |
|      17 |         250 |       00:10:36 |       87.50% |       0.3406 |      1.0000e-04 |
|      20 |         300 |  

### Test
Set up test data

In [8]:
rootFolder = 'cifar10Test';
testDS = imageDatastore(fullfile(rootFolder, categories), 'LabelSource', 'foldernames');
testDS.ReadFcn = @readFunctionTrain;

### Test classifer

In [12]:
[labels,err_test] = classify(convnet, testDS, 'MiniBatchSize', 64);

### Determine overall accuracy

In [13]:
confMat = confusionmat(testDS.Labels, labels);
confMat = confMat./sum(confMat,2);
mean(diag(confMat))


ans =

    0.8023

